### Sean Ulric C. Montano
### CPE-31
### DSP Lab Final Project: QR Code Scanner

In [1]:
import numpy as np
from pyzbar.pyzbar import decode
import cv2
from tkinter import *
from tkinter import filedialog
import webbrowser
import threading
from PIL import Image, ImageTk

class QRScannerApp:
    def __init__(self, master):
        self.master = master
        master.title("QR Scanner")

        self.detected_qr_data = set()

        self.qr_data_text = Text(master, height=10, width=50)
        self.qr_data_text.pack()

        self.select_file_button = Button(master, text="Select File", command=self.select_file)
        self.select_file_button.pack()

        self.start_button = Button(master, text="Start Video Capture", command=self.start_video_capture)
        self.start_button.pack()

        self.quit_button = Button(master, text="Quit", command=self.quit_application)
        self.quit_button.pack()

        self.capture = None
        self.video_thread = None
        self.running = False

        self.status = 0

        self.label = Label(master)
        self.label.pack()

    def select_file(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.process_image(cv2.imread(file_path))

    def start_video_capture(self):
        self.status = 1
        self.capture = cv2.VideoCapture(0) # depends on what webcam is used
        self.capture.set(3, 1280)
        self.capture.set(4, 720)

        self.running = True
        self.video_thread = threading.Thread(target=self.capture_and_process)
        self.video_thread.start()

    def capture_and_process(self):
        while self.running:
            success, frame = self.capture.read()
            if not success:
                print("Failed to capture frame")
                break

            self.process_image(frame)

            key = cv2.waitKey(1)
            if key == ord('q'):
                self.running = False

        self.capture.release()
        cv2.destroyAllWindows()

    def process_image(self, image):
        # Keep track of previously detected QR codes
        detected_qr_codes = set()

        # Draw outlines and text for previously detected QR codes
        for data in self.detected_qr_data:
            for barcode in decode(image):
                if barcode.data.decode('utf-8') == data:
                    detected_qr_codes.add(data)

                    # Outline the QR code
                    points = np.array([barcode.polygon], np.int32)
                    points = points.reshape((-1, 1, 2))
                    cv2.polylines(image, [points], True, (0, 255, 0), 3)
                    
                    # Text to show link or data
                    points2 = barcode.rect
                    cv2.putText(image, data, (points2[0], points2[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Detect and draw outlines and text for newly detected QR codes
        for barcode in decode(image):
            data = barcode.data.decode('utf-8')
            if data not in detected_qr_codes:
                detected_qr_codes.add(data)
                self.detected_qr_data.add(data)

                # Outline the QR code
                points = np.array([barcode.polygon], np.int32)
                points = points.reshape((-1, 1, 2))
                cv2.polylines(image, [points], True, (0, 255, 0), 3)

                # Text to show link or data
                points2 = barcode.rect
                cv2.putText(image, data, (points2[0], points2[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Make the link clickable
                self.qr_data_text.tag_config(f"hyper_{data}", foreground="deep sky blue", underline=1)
                self.qr_data_text.tag_bind(f"hyper_{data}", "<Button-1>", lambda e, url=data: self.callback(url))
                self.qr_data_text.insert(END, data, f"hyper_{data}")
                self.qr_data_text.insert(END, "\n\n")

        # Convert the image to RGB format for display
        image_bgr = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # # Convert the image to PIL format
        image_pil = Image.fromarray(image_bgr)

        # # Convert the PIL image to Tkinter format"
        photo = ImageTk.PhotoImage(image_pil)

        # Update the label with the new image
        if self.status == 1:
            self.label.configure(image=photo)
            self.label.image = photo

    def callback(self, url):
        webbrowser.open_new_tab(url)

    def quit_application(self):
        self.running = False
        if self.capture:
            self.capture.release()
        cv2.destroyAllWindows()
        self.master.quit()

def main():
    root = Tk()
    app = QRScannerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
